In [ ]:
# Libraries used in calculations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
from scipy.stats import linregress # used to get the linear parameters from the results
import copy

In [ ]:
# Parameters
T = 1 # Time
sig = 0.5 # sigma(0)
alpha = 0.01 # # alpha in SABR
F_0 = 1 # initial forward price, F(0)
rho = 0.25 # correlation between Brownian motions

In [ ]:
# SABR function calculates pahs for the F(t), EM discretisation and Milstein discretisation
def SABR(T, N, sig_0, M, alpha, F_0, rho):
    # There will be M Monte Carlo simulations
    dt = T/N # step size
    F_T = np.ones(M)*F_0 # Array for F(T) values
    EM = np.ones(M)*F_0 # Array for EM values
    Mil = np.ones(M)*F_0 # Array for Milstein values
    sig_t = np.ones(M)*sig_0 # Array for sigma(t)
    A = np.array([alpha*rho, alpha*np.sqrt(1 - rho**2)]) # cholesky factor
    # iterate through N steps, calculating for M paths 
    for i in range(N):
        Z = np.sqrt(dt) * np.random.normal(0, 1, (2, M)) # Browian increment for B1 and B2
        B2 = A[0]*Z[0,:] + A[1]*Z[1,:]  # Brownian motion for B1 in the SABR model
        B1 = Z[0,:] # Brownian motion for B1 in the SABR model
        sig_t = sig_t*np.exp((-alpha**2 / 2)*dt + B2) # sigma(t)
        
        F_T = F_T*np.exp((-sig_t**2 / 2)*dt + sig_t*B1) # exact solution
        EM = EM + EM * sig_t * B1 # EM approximation
        Mil = Mil + Mil * sig_t * B1 + 0.5 * (sig_t**2) * Mil * (B1**2 - dt) # Milstein approximation
    return F_T, Mil, EM # return values when n=N

# Strong Error

In [ ]:
strongError = pd.DataFrame(columns=['EM', 'Mil', 'N']) # Dataframe to hold strong error
Nsteps = np.array([2**6, 2**8, 2**9, 2**10, 2**12]) # Numbers of steps evaluated
err = np.zeros([1, 2]) # placeholder for error
for i in range(len(Nsteps)):
        F_T, Mil, EM = SABR(T=T, N=Nsteps[i], M=10**4, sig_0=sig, alpha=alpha, F_0=F_0, rho=rho) # calculate paths using SABR function
        err_EM = np.mean(np.abs(EM - F_T)) # Strong error for EM at Ni
        err_Mil = np.mean(np.abs(Mil - F_T)) # Strong error for Milstein at Ni
        # Add strong error to Dataframe
        strongError = strongError.append({
            'EM':err_EM,
            'Mil':err_Mil,
            'N':int(Nsteps[i])
        }, ignore_index=True)
strongError.index = strongError.N
del strongError['N']

In [ ]:
# Plot Strong error and see if there is strong convergence
Dt = T / Nsteps
plt.figure(figsize=(10, 10))
plt.loglog(Dt, strongError.iloc[:,0],'o', markersize = 10, linestyle = '-', label='EM') # EM strong error
plt.loglog(Dt, strongError.iloc[:,1],'x', markersize = 10, linestyle = '-', label='Milstein') # Milstein strong error
plt.loglog(Dt,Dt, '--', label='ref') # reference line, with slope=1
plt.xlabel('h', fontsize=18)
plt.ylabel('Sample average of |F(t) - Fn|', fontsize=18)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.legend()
# plt.savefig('Strong error convergence for SABR.jpg')

In [ ]:
# calculate order of convergence, gamma, and C
# use linregress from scipy
EM_gamma, EM_C = linregress(np.log(Dt), np.log(strongError.iloc[:,0]))[0:2]
Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(strongError.iloc[:,1]))[0:2]
C_EM = m.exp(EM_C)
C_Mil = m.exp(Mil_C)
print('For the Euler-Maruyama method, gamma = {} and C = {}'.format(EM_gamma, C_EM))
print('For the Milstein method, gamma = {} and C = {}'.format(Mil_gamma, C_Mil))

# Parameter Analysis

## $\alpha$ analysis

In [ ]:
# Parameters, same meaning as above
T = 1
sig = 0.5
F_0 = 1
rho = 0.25

In [ ]:
alpha_array = np.array([0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]) # alpha values to evaluate
Nsteps = np.array([2**6, 2**8, 2**9, 2**10, 2**12])
err = np.zeros((len(Nsteps), len(alpha_array)))
for j in range(len(alpha_array)):
    for i in range(len(Nsteps)):
            F_T, Mil = SABR(T=T, N=Nsteps[i], M=10**3, sig_0=sig, alpha=alpha_array[j], F_0=F_0, rho=rho)
            err[i, j] = np.mean(np.abs(Mil - F_T))

In [ ]:
Dt = T / Nsteps
gamma_C = np.zeros((len(alpha_array), 2))
for i in range(len(alpha_array)):
    Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(err[:,i]))[0:2]
    C_Mil = m.exp(Mil_C)
    gamma_C[i,0] = Mil_gamma
    gamma_C[i,1] = C_Mil
    print('For the Milstein method, gamma = {} and C = {}'.format(Mil_gamma, C_Mil))

In [ ]:
plt.figure(figsize=(12.5,7.5))
plt.scatter(alpha_array[:-1], gamma_C[:-1,0], s=gamma_C[:-1,1]*10000)
plt.xlabel(r'$\alpha$', fontsize=22)
plt.ylabel('Order of Convergence', fontsize=18)
plt.grid()
# plt.savefig('C:\\Users\\chris\\Documents\\Financial and computational maths\\MF6016 Dissertation\\final report\\figures\\SABR_alpha_convergenceAffect.jpg')


## $\rho$ analysis

In [ ]:
# parameters
T = 1
sig = 0.5
alpha = 0.01
F_0 = 1

In [ ]:
rho_array = np.array([-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1]) # rho values to evaluate
Nsteps = np.array([2**6, 2**8, 2**9, 2**10, 2**12])
err_rho = np.zeros((len(Nsteps), len(rho_array)))
for j in range(len(rho_array)):
    for i in range(len(Nsteps)):
            F_T, Mil = SABR(T=T, N=Nsteps[i], M=10**3, sig_0=sig, alpha=alpha, F_0=F_0, rho=rho_array[j])
            err_rho[i, j] = np.mean(np.abs(Mil - F_T))

In [ ]:
Dt = T / Nsteps
rho_gamma_C = np.zeros((len(rho_array), 2))
for i in range(len(rho_array)):
    Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(err_rho[:,i]))[0:2]
    C_Mil = m.exp(Mil_C)
    rho_gamma_C[i,0] = Mil_gamma
    rho_gamma_C[i,1] = C_Mil
    print('For the Milstein method, gamma = {} and C = {}'.format(Mil_gamma, C_Mil))

In [ ]:
plt.figure(figsize=(12.5,7.5))
plt.scatter(rho_array, rho_gamma_C[:,0], s=rho_gamma_C[:,1]*50000)
plt.xlabel(r'$\rho$', fontsize=18)
plt.ylabel('Order of Convergence', fontsize=18)
plt.grid()
# plt.savefig('C:\\Users\\chris\\Documents\\Financial and computational maths\\MF6016 Dissertation\\final report\\figures\\SABR_rho_convergenceAffect.jpg')


## $\sigma (0)$ analysis

In [ ]:
# parameters
T = 1
# sig = 0.5
alpha = 0.01
F_0 = 1
rho = 0.25

In [ ]:
sig_array = np.array([0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]) # sigma(0) values to evaluate
Nsteps = np.array([2**6, 2**8, 2**9, 2**10, 2**12])
err_sig = np.zeros((len(Nsteps), len(sig_array)))
for j in range(len(sig_array)):
    for i in range(len(Nsteps)):
            F_T, Mil = SABR(T=T, N=Nsteps[i], M=10**3, sig_0=sig_array[j], alpha=alpha, F_0=F_0, rho=rho)
            err_sig[i, j] = np.mean(np.abs(Mil - F_T))

In [ ]:
Dt = T / Nsteps
sig_gamma_C = np.zeros((len(sig_array), 2))
for i in range(len(sig_array)):
    Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(err_sig[:,i]))[0:2]
    C_Mil = m.exp(Mil_C)
    sig_gamma_C[i,0] = Mil_gamma
    sig_gamma_C[i,1] = C_Mil
    print('For the Milstein method, gamma = {} and C = {}'.format(Mil_gamma, C_Mil))

In [ ]:
plt.figure(figsize=(12.5,7.5))
plt.scatter(sig_array, sig_gamma_C[:,0], s=sig_gamma_C[:,1]*50000, alpha=0.5)
plt.xlabel(r'$\sigma (0)$', fontsize=18)
plt.ylabel('Order of Convergence', fontsize=18)
plt.grid()
# plt.savefig('C:\\Users\\chris\\Documents\\Financial and computational maths\\MF6016 Dissertation\\final report\\figures\\SABR_sigma0_convergenceAffect.jpg')


## F(0) analysis

In [ ]:
# parameters
T = 1
sig = 0.5
alpha = 0.01
rho = 0.25

In [ ]:
F0_array = np.array([1, 5, 10, 50, 100, 500, 1000]) # F(0) values to evaluate
Nsteps = np.array([2**6, 2**8, 2**9, 2**10, 2**12])
err_F0 = np.zeros((len(Nsteps), len(F0_array)))
for j in range(len(F0_array)):
    for i in range(len(Nsteps)):
            F_T, Mil = SABR(T=T, N=Nsteps[i], M=10**3, sig_0=sig, alpha=alpha, F_0=F0_array[j], rho=rho)
            err_F0[i, j] = np.mean(np.abs(Mil - F_T))

In [ ]:
Dt = T / Nsteps
F0_gamma_C = np.zeros((len(F0_array), 2))
for i in range(len(F0_array)):
    Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(err_F0[:,i]))[0:2]
    C_Mil = m.exp(Mil_C)
    F0_gamma_C[i,0] = Mil_gamma
    F0_gamma_C[i,1] = C_Mil
    print('For the Milstein method, gamma = {} and C = {}'.format(Mil_gamma, C_Mil))

In [ ]:
plt.figure(figsize=(12.5,7.5))
plt.scatter(F0_array, F0_gamma_C[:,0], s=F0_gamma_C[:,1]*1000, alpha=0.5)
plt.xlabel('F(0)', fontsize=18)
plt.ylabel('Order of Convergence', fontsize=18)
plt.gca().set_xscale('log')
plt.grid()
# plt.savefig('C:\\Users\\chris\\Documents\\Financial and computational maths\\MF6016 Dissertation\\final report\\figures\\SABR_F0_convergenceAffect.jpg')
